## SNU PPSML - Machine Learning Exercise 3 (Note10)

#### **Contents**
---
* ML exercise 1: Gradient Descent [Optimization @ ML] (HW05)  

---
* ML exercise 2: Neural Network [Representation @ ML] (실습과제 11/5-8 & 11/9 강의, **NNfactory.py**) 
    - 입력 속성데이터와 지도라벨값 $\{(x_i, y_i)\}$ @ 인코딩: 입력과 출력층의 설정  

    - 순전파 신경망(def feedforward())의 구성을 위한 여러가지 파라메터들     
        - $N_{layers}$ & $N_{nodes}$ in each layer
        - 가중치와 편향치 ($W$ & $B$)
        - 가중합(퍼셉트론 입력) : 
        
        > $a_j=W_{ji} f_i(a_i)$ (전층의 출력 $f_i$에 대한 가중합)
        
        - 활성화 함수(출력) : $f_j(a_j)$
        
        > $f(a)=\frac{1}{1+\exp^{-a}}$ for sigmoid  
        
        > $f(a)=\tanh(a)$ for tanh  
        
        > $f(a_k)=\frac{\exp[a_k]}{\sum_{k'}\exp[a_{k'}]}$ for softmax  
        
        > $f(a)=a$ for $ a > 0$ otherwise $0$ (ReLU)
        
        - 데이터의 순전파를 통한 최종 출력값 얻기: def feedforward(input_features)
        - ...
        
    - 오차보정의 역전파:  
    
        - 지도라벨값의 인코딩과 오차함수($E(w;x)$)의 정의
        - $\delta_j$ (가중합 $a_j$에 대한 오차보정항 $\equiv\frac{\partial E}{\partial a_j}$)
        - $\delta_j$를 통한 가중치보정 역전파의 구현: def backpropagate(target_label)
        - ...
        
    - 많은 데이터에 대한 학습 알고리즘 구현
    - check a contour of MLP's probability output for classification of 2D data  
    
---   
* **ML exercise 3: Training a NN for Regression** & Classification [**Evaluation**, Rep, Opt @ ML] (HW06-07, note09-10)
    - batch GD, mini-batch GD, stochastic GD [HW07]  
        - Visualize the minimizers in 2D
    - Validation of model, Over-fitting, Bias & Variance**  
        - Visualize an over-fitted status [note09]
    - Evaluation of model performance
        - error(loss), accuracy (...) --> learning curve [HW06]
        - NN score & ROC(Receiver Operating Characteristic) curve [note09, HW06]
    - **Training a NN for Regression [note10]**

In [0]:
# Printing all outputs in a cell (not only the last output)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

----
### ** 3. Evaluation @ ML ** 

### For Supervised Learning (note10)

1) batch gradient descent, mini-batch GD, stochastic GD (HW07)

2) Validation of model
    - [과적합 (Over-fitting)상태의 확인] (note09)

3) Evaluation of model performance      
    - [error/loss function] & [accuracy] (HW06)
    -  implement new error functions in your NN class  
       ('binary cross-entropy' & 'categorical cross-entropy') (HW07)

    - [neural network score] (note09)
    - [ROC (Receiver Operating Characteristic) curve] & [its AUC (area under curve)] (note09, HW06)
     
**4) Regression **  
   1. training of a regression model
   2. validation of the regression model (과적합 상태의 생성 및 확인)

---
#### ** 4) 회귀 (Regressions)** 

**1. Training of a regression model with 1D data $x$ with real number label $y$, $x\rightarrow y(x)$**

   - 주어진 1차원의 속성공간에 정의된 데이터 $x_i$들이 어떤 (연속/불연속) 실수값 $y_i$에 대응되고 있다고 하자.
       > $x_i \, \rightarrow \, y_i\, in\, {\mathbb R}$
       
     이때 $x_i \, \rightarrow \, f(x_i;W_{trained}) = y_i$가 되도록, (신경망의/기계의) 출력모형 $f$를 데이터로부터 학습해내는 문제를 회귀(Regression)문제라 하며, 이는 분류문제(classification, $y_i$가 한정된 수량의 불연속값인 경우)와 더불어 머신러닝 지도학습(supervised learning)이 주요한 한 갈래가 된다. 

   - 회귀모형에서는 출력층의 활성화함수를 항등원 $f(a)=a$를 사용한다. NNfactory.MLP클래스를 활용하여 다음의 데이터를 학습해내는 신경망 회귀모형을 건설해보자.

   - 훈련한 모형의 예측치를 matplotlib를 활용하여 시각화해보자.


In [0]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# 데이터 만들기 (속성 X)
X = np.linspace(0,1,51)

# 데이터 만들기 (라벨 Y(for X) in real number)
a, b = 1., 0.001
Y_train = a*np.sin(2*np.pi*X) + b*np.random.randn(len(X))
Y_test = a*np.sin(2*np.pi*X) + b*np.random.randn(len(X))

# reshape of X & Y for making main data block
X=X.reshape((len(X),1))
Y_train=Y_train.reshape((len(X),1))
Y_test=Y_test.reshape(len(X),1)

# stacking two columns - Y, X, in to data_train/data_test
data_train = np.c_[Y_train,X]
data_test = np.c_[Y_test,X]

# data check [y(x_i), x_i] 
data_train[:10]
# data[0]
data_test[:10]

array([[ -2.60706411e-04,   0.00000000e+00],
       [  1.26486057e-01,   2.00000000e-02],
       [  2.49667007e-01,   4.00000000e-02],
       [  3.68825971e-01,   6.00000000e-02],
       [  4.81271148e-01,   8.00000000e-02],
       [  5.87226664e-01,   1.00000000e-01],
       [  6.84140222e-01,   1.20000000e-01],
       [  7.70862658e-01,   1.40000000e-01],
       [  8.44046913e-01,   1.60000000e-01],
       [  9.04478607e-01,   1.80000000e-01]])

In [0]:
# 데이터 시각화

fig1 = plt.figure(figsize=(8,4))
ax1 = fig1.add_subplot(111)

ax1.plot(data_train[:,1],data_train[:,0], 'r.')
ax1.plot(data_test[:,1],data_test[:,0], 'b.')
ax1.set_xlabel('x', fontsize=20)
ax1.set_ylabel('$y(x)$', fontsize=20)
ax1.grid(True)

plt.show()

In [0]:
# 신경망 클래스 로딩
%reload_ext autoreload
%autoreload 2

import NNfactory

In [0]:
model_str = '1:identity|'+1*'3:tanh|'+'1:identity'
lr = 0.001
name_tag = 'regression_lr'+str(lr)

In [0]:
mynn = NNfactory.MLP(model_structure=model_str, \
                     model_nametag=name_tag, \
                     learning_rate=lr, \
                     encoding='float')


 * 다음과 같은 구조의 다층퍼셉트론 연결이 초기화 되었습니다 *

 > 모델이름 = regression_lr0.001
 > 총 층수 (입력 + 은닉(s) + 출력) =  3
 > 각 층에서의 노드수 =  [1, 3, 1]
 > 각 층에서의 활성화 함수 =  ['identity', 'tanh', 'identity']
 > 학습률(Learning Rate) =  0.001
 > 지도라벨 인코딩 방식 =  float


In [0]:
# 훈련 준비
data_type=None #mnist' # None
encoding='float' #'one-hot' #'integer' # 'float'

# 훈련 & 테스트 데이터 준비
n_data_max=100  # 훈련에 사용할 데이터 갯수 (max = 10000)
n_data_test=1000  # 테스트에 사용할 테이터 갯수 (10000 - n_data_max)

## 훈련 데이터
training_data_list = data_train[:n_data_max] # 

## 테스트 데이터
test_data_list = data_test[-n_data_test:]

# 최대학습주기 설정
epochs = 50000

In [0]:
# 설정된 최대학습주기동안 훈련

for e in range(epochs):
    
    id_data = 0
    
    for data in training_data_list[:n_data_max]:
        
        # 프로세스 게이지
        id_data += 1
        
        # 입력/지도 데이터 가공 
        if data_type == 'mnist':
            # split the mnist data by the ',' commas
            all_list = data.split(',')
            # 입력 속성 데이터 스케일링 (preprocessing)
            input_list = np.asfarray(all_list[1:])/255.0 #* 0.99) + 0.01
            # 지도 라벨 벡터 가공 (shape = (10,))
            target_list = np.zeros(10) #mynn.n_nodes[-1])
            # all_values[0] is the target label for this data
            target_list[int(all_list[0])] = 1.0
            
        else:
        
            input_list = data[1:] #np.asfarray(all_values[1:])        
            target_origin = data[0]
            
            if encoding == 'one-hot':

                target_list = np.zeros(mynn.n_nodes[-1])
                target_list[int(data[0])] = 1
    
            elif encoding == 'integer':
                
                target_list = np.zeros(1)
                target_list[0] = int(data[0])

            elif encoding == 'float':
                
                target_list = np.zeros(1)
                target_list[0] = data[0]
                
            else:
                raise ValueError(' => check your encoding scheme. ')
                
        mynn.train(input_list, target_list)
        
        pass

    if (e%1000==0):
        print (' --------------------------------------')
        print(' * epoch = {}'.format(e+1))
        print(' > 훈련 샘플에 대한 성능 (정확도 & 평균에러) ')
        mynn.check_accuracy_error(training_data_list, 0, n_data_max-1, data_type=None)
        print('')
        print(' > 테스트 샘플에 대한 성능 (정확도 & 평균에러) ')
        mynn.check_accuracy_error(test_data_list, 0, n_data_test-1, data_type=None)
    
    
    pass

---
* 훈련한 모형의 저장

훈련된 모형은 NNfactory클래스 안의 save_model메소드를 사용하여 .npy포맷의 numpy array로 저장할 수 있다.

In [0]:
mynn.save_model(fname='mlp_reg_1tanh_epoch200k.npy', nametag='reg_1tanh')


---
* 저장된 모형 불러오기 :

저장된 .npy파일로부터 신경망정보가 담긴 넘파이 배열을 직접 로드하고, 이 넘파이 배열을 새 신경망 인스턴스 생성에 사용하여 저장된 모형과 똑같은 신경망을 로드한다.

In [0]:
mynn_npy = np.load('mlp_reg_1tanh_epoch200k.npy')
mynn_load = NNfactory.MLP(load_model_np=mynn_npy)


 * 다음과 같은 구조의 다층퍼셉트론 모형이 "load_model_np" 정보로부터 로드되었습니다. *

 > 모델이름 = reg_1tanh
 > 총 층수 (입력 + 은닉(s) + 출력) =  3
 > 각 층에서의 노드수 =  [1, 3, 1]
 > 각 층에서의 활성화 함수 =  ['identity', 'tanh', 'identity']
 > 학습률(Learning Rate) =  0.001
 > 지도라벨 인코딩 방식 =  float


---
* 학습한 회귀 모형을 시각화해보자

In [0]:
# data_train에 대한 회귀모형의 출력값을 얻고 저장.
result=[]
for data in data_train:
    result.append(mynn_load.feedforward(data[1:])[0])
   

<class 'numpy.ndarray'>
(51, 2)


---
* 잠깐!. 심심하니 학습한 회귀 모형을 분해해보자 

In [0]:
# 신경망 각 노드의 입력값
mynn_load.a
# 신경망 각 노드의 출력값
mynn_load.f
# 신경망 연결 가중치값
mynn_load.W
# 신경망 편향값
mynn_load.B


[array([[ 1.]]), array([[-0.09474994],
        [-2.05878377],
        [-1.53500404]]), array([[ 0.00640182]])]

In [0]:
# 최종 은닉층 3개 (1|3|1)의 출력값과 가중치를 가지고 최종 출력노드의 출력값을 재구성
f=mynn_load.f[1].reshape(3)
w=mynn_load.W[1].reshape(3)
b=mynn_load.B[1]

# 재구성한 출력값 1
y = np.dot(f,w) + b
print(y)

# 현재 인스턴스가 가진 출력값
print(mynn_load.f[2])

In [0]:
# 심심하면.. 
# 위의 예제에서
# 모든 데이터에 대한 출력을 다음의 4파트(가중치가 곱해진 은닉층의 각 노드출력 & 편향값, 각각의 출력모형)로 구분하여, 
# 각각의 출력값을 역시 그래프(f' vs X)로 그려보고, 
# 어떤 함수들이 어떤 가중치로 곱해져서,저런 훌륭한 회귀모형이 되었는지 감상해보자. 
# y[x] = f1.W1[x] + f2.W2[x] + f3.w3[x] + b[x]

#### ** 4) [회귀(Regressions)]** 

**2. Validation of a Regression Model**

   - 1의 회귀모형 훈련시에 데이터의 양을 줄이고 (&노이즈 추가), 동시에 신경망 모형의 표현능력을 키워서, 과적합 상태(overfitting)를 만들어보고 이를 시각화해보자.



In [0]:
# 데이터 만들기

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Daa (X) generation
# X_train = np.linspace(0,1,10)
X_train = np.random.uniform(0,1,20)
X_train.sort()
X_test = np.random.uniform(0,1,10)
X_test.sort()

# Label (=Y(X)) generation
a, b = 1., 0.2
Y_train = a*np.sin(2*np.pi*X_train) + b*np.random.randn(len(X_train))
Y_test = a*np.sin(2*np.pi*X_test) + b*np.random.randn(len(X_test))

# reshape of X & Y for making main data block
X_train=X_train.reshape((len(X_train),1))
X_test=X_test.reshape((len(X_test),1))

Y_train=Y_train.reshape((len(X_train),1))
Y_test=Y_test.reshape(len(X_test),1)

# stacking two columns - Y and X, into data_train/test array 
data_train = np.c_[Y_train,X_train]
data_test = np.c_[Y_test,X_test]

# train data check [y(x_i), x_i] 
data_train[:10]
# test data check 
# data_test[:10]

# # data save
# np.save('data_reg_overfit_train.npy',data_train)
# np.save('data_reg_overfit_test.npy',data_test)

# # data load from file
# data_train = np.load('data_reg_overfit_train.npy')
# data_test= np.load('data_reg_overfit_test.npy')


In [0]:
## 데이터 시각화
fig1 = plt.figure(figsize=(8,4))
ax1 = fig1.add_subplot(111)

ax1.plot(data_train[:,1],data_train[:,0], 'r.')
ax1.plot(data_test[:,1],data_test[:,0], 'b.')
ax1.set_xlabel('x', fontsize=20)
ax1.set_ylabel('$y(x)$', fontsize=20)
ax1.grid(True)

plt.show()

In [0]:
%reload_ext autoreload
%autoreload 2

import NNfactory

In [0]:
model_str = '1:identity|'+6*'100:relu|'+'1:identity'
lr = 0.001
name_tag = 'regression_lr'+str(lr)

In [0]:
mynn = NNfactory.MLP(model_structure=model_str, \
                     model_nametag=name_tag, \
                     learning_rate=lr, \
                     encoding='float')

In [0]:
# 훈련 준비
data_type=None #mnist' # None
encoding='float' #'one-hot' #'integer' # 'float'

# 훈련 & 테스트 데이터 준비
n_data_max=20  # 훈련에 사용할 데이터 갯수 (max = 10000)
n_data_test=20  # 테스트에 사용할 테이터 갯수 (10000 - n_data_max)

## 훈련 데이터
training_data_list = data_train[:n_data_max] # 

## 테스트 데이터
test_data_list = data_test[-n_data_test:]

# 최대학습주기 설정
epochs = 50000

In [0]:
# 설정된 최대학습주기동안 훈련

for e in range(epochs):
    
    id_data = 0
    
    for data in training_data_list[:n_data_max]:
        
        # 프로세스 게이지
        id_data += 1
        
        # 입력/지도 데이터 가공 
        if data_type == 'mnist':
            # split the mnist data by the ',' commas
            all_list = data.split(',')
            # 입력 속성 데이터 스케일링 (preprocessing)
            input_list = np.asfarray(all_list[1:])/255.0 #* 0.99) + 0.01
            # 지도 라벨 벡터 가공 (shape = (10,))
            target_list = np.zeros(10) #mynn.n_nodes[-1])
            # all_values[0] is the target label for this data
            target_list[int(all_list[0])] = 1.0
            
        else:
        
            input_list = data[1:] #np.asfarray(all_values[1:])        
            target_origin = data[0]
            
            if encoding == 'one-hot':

                target_list = np.zeros(mynn.n_nodes[-1])
                target_list[int(data[0])] = 1
    
            elif encoding == 'integer':
                
                target_list = np.zeros(1)
                target_list[0] = int(data[0])

            elif encoding == 'float':
                
                target_list = np.zeros(1)
                target_list[0] = data[0]
                
            else:
                raise ValueError(' => check your encoding scheme. ')
                
        mynn.train(input_list, target_list)
        
        pass

    if (e%1000==0):
        print (' --------------------------------------')
        print(' * epoch = {}'.format(e+1))
        print(' > 훈련 샘플에 대한 성능 (정확도 & 평균에러) ')
        mynn.check_accuracy_error(training_data_list, 0, n_data_max-1, data_type=None)
        print('')
        print(' > 테스트 샘플에 대한 성능 (정확도 & 평균에러) ')
        mynn.check_accuracy_error(test_data_list, 0, n_data_test-1, data_type=None)
    
    
    pass

---
* 훈련한 모형의 저장

훈련된 모형은 NNfactory클래스 안의 save_model메소드를 사용하여 .npy포맷의 numpy array로 저장할 수 있다.

In [0]:
mynn.save_model(fname='mlp_reg_overfit.npy', nametag='reg_overfit')


---
* 저장된 모형 불러오기 :

저장된 .npy파일로부터 신경망정보가 담긴 넘파이 배열을 직접 로드하고, 이 넘파이 배열을 새 신경망 인스턴스 생성에 사용하여 저장된 모형과 똑같은 신경망을 로드한다.

In [0]:
mynn_npy = np.load('mlp_reg_overfit.npy')
mynn_load = NNfactory.MLP(load_model_np=mynn_npy)

---
* 과적합 상태의 회귀 모형을 시각화해보기

In [0]:
# 과적합 상태의 모형 출력 확인을 위한 잘 정렬된 테스트 데이터 (X_val) 정의하기
X_val = np.linspace(0,1,1000)
X_val = X_val.reshape((len(X_val),1))

# X_val 데이터에 대한 회귀모형 출력값 얻기
result=[]
for X in X_val:
    result.append(mynn_load.feedforward(X)[0])   

In [0]:
## plot objects 
fig1 = plt.figure(figsize=(8,4))
ax1 = fig1.add_subplot(111)

# 과적합 상태의 모형의 훈련에 쓰인 훈련데이터 플랏
ax1.plot(data_train[:,1],data_train[:,0], 'g*',label='train data')

# 과적합 모형의 출력을 X_val를 사용하여 플랏
ax1.plot(X_val,result, 'b-',label='regression output')

ax1.set_xlabel('x', fontsize=20)
ax1.set_ylabel('$y(x)$', fontsize=20)
ax1.legend(loc='best')
ax1.grid(True)

plt.show()